# Zestaw 4.1 - "Podzapytania - ćwiczenia końcowe"

In [ ]:
USE Northwind

## Ćwiczenie #1

**1.1 Wybierz nazwy i numery telefonów klientów , którym w 1997 roku przesyłki
dostarczała firma United Package**

In [ ]:
SELECT CompanyName, Phone
FROM Customers
WHERE CustomerID IN
    (SELECT DISTINCT CustomerID
    FROM Orders
    WHERE ShipVia IN
        (SELECT ShipperID
        FROM Shippers
        WHERE CompanyName = 'United Package'))

**1.2 Wybierz nazwy i numery telefonów klientów, którzy kupowali produkty z kategorii
Confections**

In [ ]:
SELECT C.CompanyName, C.Phone
FROM Customers AS C
WHERE C.CustomerID IN
    (SELECT O.CustomerID
    FROM Orders AS O
    WHERE O.OrderID IN
        (SELECT OD.OrderID
        FROM [Order Details] AS OD
        WHERE OD.ProductID IN
            (SELECT P.ProductID 
            FROM Products AS P
            WHERE P.CategoryID IN 
                (SELECT CA.CategoryID
                FROM Categories AS CA
                WHERE CA.CategoryName = 'Confections'))))

In [ ]:
SELECT DISTINCT C.CompanyName, C.Phone, CA.CategoryName
FROM Customers AS C
INNER JOIN Orders AS O
ON C.CustomerID = O.CustomerID
INNER JOIN [Order Details] AS OD
ON OD.OrderID = O.OrderID
INNER JOIN Products AS P
ON P.ProductID = OD.ProductID
INNER JOIN Categories AS CA
ON CA.CategoryID = P.CategoryID
WHERE CA.CategoryName = 'Confections'

**1.3 Wybierz nazwy i numery telefonów klientów, którzy nie kupowali produktów z
kategorii Confections**

In [ ]:
SELECT C.CompanyName, C.Phone
FROM Customers AS C
WHERE C.CustomerID NOT IN
    (SELECT O.CustomerID
    FROM Orders AS O
    INNER JOIN [Order Details] AS OD
    ON O.OrderID = OD.OrderID
    INNER JOIN Products AS P
    ON OD.ProductID = P.ProductID
    INNER JOIN Categories AS CA
    ON CA.CategoryID = P.CategoryID
    WHERE CA.CategoryName = 'Confections')

In [ ]:
SELECT Customers.CompanyName, Customers.Phone FROM Customers
LEFT OUTER JOIN 
    (Orders AS O INNER JOIN [Order Details] AS OD
    ON O.OrderID = OD.OrderID
    INNER JOIN products AS P
    ON P.productID = OD.ProductID 
    INNER JOIN categories AS C
    ON C.CategoryID = P.CategoryID AND C.CategoryName = 'Confections' )
ON Customers.CustomerID = O.customerID 
WHERE C.CategoryID IS NULL

## Ćwiczenie #2
**2.1 Dla każdego produktu podaj maksymalną liczbę zamówionych jednostek**

In [ ]:
SELECT DISTINCT OD.OrderID, OD.Quantity
FROM [Order Details] AS OD
WHERE OD.Quantity = 
    (SELECT MAX(Quantity)
    FROM [Order Details] AS OD2
    WHERE OD.OrderID = OD2.OrderID)
ORDER BY OD.Quantity


In [ ]:
SELECT OD.OrderID, MAX(OD.Quantity)
FROM [Order Details] AS OD
GROUP BY OD.OrderID

**2.2 Podaj wszystkie produkty których cena jest mniejsza niż średnia cena produktu**

In [ ]:
SELECT P.ProductID
FROM Products AS P
WHERE P.UnitPrice < 
    (SELECT AVG(P2.UnitPrice)
    FROM Products AS P2)

**2.3 Podaj wszystkie produkty których cena jest mniejsza niż średnia cena produktu
danej kategorii**

In [ ]:
SELECT P.ProductID 
FROM Products AS P
WHERE P.UnitPrice <
    (SELECT AVG(P2.UnitPrice)
    FROM Products AS P2
    INNER JOIN Categories AS C
    ON P2.CategoryID = C.CategoryID
    WHERE C.CategoryID = P.CategoryID)

## Ćwiczenie #3
**3.1 Dla każdego produktu podaj jego nazwę, cenę, średnią cenę wszystkich
produktów oraz różnicę między ceną produktu a średnią ceną wszystkich
produktów**

In [ ]:
SELECT *, A.UnitPrice - A.AvgPrice AS Diff
FROM (
    SELECT ProductName, UnitPrice, (
        SELECT AVG(UnitPrice) FROM Products
    ) AS AvgPrice
    FROM Products
) AS A

**3.2 Dla każdego produktu podaj jego nazwę kategorii, nazwę produktu, cenę, średnią
cenę wszystkich produktów danej kategorii oraz różnicę między ceną produktu a
średnią ceną wszystkich produktów danej kategorii**

In [ ]:
SELECT C.CategoryName, P.ProductName, P.UnitPrice, (
    SELECT AVG(P2.UnitPrice)
    FROM Products AS P2
    WHERE P2.CategoryID = P.CategoryID
) AS AvgCatPrice, P.UnitPrice-(
    SELECT AVG(P2.UnitPrice)
    FROM Products AS P2
    WHERE P2.CategoryID = P.CategoryID
) AS Diff
FROM Products AS P
INNER JOIN Categories AS C
ON P.CategoryID = C.CategoryID

In [ ]:
SELECT *, A.UnitPrice-A.CatAvgPrice AS Diff
FROM (
    SELECT C.CategoryName, P.ProductName, P.UnitPrice, (
        SELECT AVG(P2.UnitPrice)
        FROM Products AS P2
        WHERE P2.CategoryID = P.CategoryID
    ) AS CatAvgPrice
    FROM Products AS P
    INNER JOIN Categories AS C
    ON P.CategoryID = C.CategoryID
) AS A

## Ćwiczenie #4
**4.1 Podaj łączną wartość zamówienia o numerze 1025 (uwzględnij cenę za przesyłkę)**

In [ ]:
SELECT O.OrderID, (
    SELECT SUM(OD.UnitPrice * OD.Quantity * (1-OD.Discount))
    FROM [Order Details] AS OD
    WHERE OD.OrderID = O.OrderID
) + O.Freight AS TotalPrice
FROM Orders AS O
WHERE O.OrderID = 10250

**4.2 Podaj łączną wartość zamówień każdego zamówienia (uwzględnij cenę za
przesyłkę)**

In [ ]:
SELECT O.OrderID, (
    SELECT SUM(OD.UnitPrice * OD.Quantity * (1-OD.Discount))
    FROM [Order Details] AS OD
    WHERE OD.OrderID = O.OrderID
) + O.Freight AS TotalPrice
FROM Orders AS O

**4.3 Czy są jacyś klienci którzy nie złożyli żadnego zamówienia w 1997 roku, jeśli tak
to pokaż ich dane adresowe**

In [ ]:
SELECT CompanyName, Address, City, Country
FROM Customers
WHERE CustomerID NOT IN (
    SELECT CustomerID
    FROM Orders
    WHERE YEAR(OrderDate) <> 1997 
)

**4.4 Podaj produkty kupowane przez więcej niż jednego klienta**

In [ ]:
SELECT P.ProductName
FROM Products AS P
WHERE (
    SELECT COUNT(O.CustomerID)
    FROM [Order Details] AS OD
    INNER JOIN Orders AS O
    ON OD.OrderID = O.OrderID
    WHERE OD.ProductID = P.ProductID
    GROUP BY OD.ProductID
) > 1

## Ćwiczenie #5
**5.1 Dla każdego pracownika (imię i nazwisko) podaj łączną wartość zamówień
obsłużonych przez tego pracownika (przy obliczaniu wartości zamówień
uwzględnij cenę za przesyłkę**

In [ ]:
SELECT E.FirstName, E.LastName, (
    SELECT O.Freight + (
        SELECT OD.UnitPrice * OD.Quantity * (1-OD.Discount) AS Total
        FROM [Order Details] AS OD
        WHERE O.OrderID = OD.OrderID
    )
    FROM Orders AS O
    WHERE O.EmployeeID = E.EmployeeID
) AS TotalPrice
FROM Employees AS E
--nie działa

In [ ]:
SELECT E.FirstName, E.LastName, 
    ROUND(SUM(O.Freight + OD.UnitPrice * OD.Quantity * (1-OD.Discount)), 2) AS TotalServed
FROM Employees AS E
INNER JOIN Orders AS O
ON O.EmployeeID = E.EmployeeID
INNER JOIN [Order Details] AS OD
ON OD.OrderID = O.OrderID
GROUP BY E.FirstName, E.LastName

**5.2 Który z pracowników obsłużył najaktywniejszy (obsłużył zamówienia o
największej wartości) w 1997r, podaj imię i nazwisko takiego pracownika**

In [ ]:
SELECT TOP 1 E.FirstName, E.LastName
FROM Employees AS E
INNER JOIN Orders AS O
ON O.EmployeeID = E.EmployeeID AND YEAR(O.OrderDate) = 1997
INNER JOIN [Order Details] AS OD
ON OD.OrderID = O.OrderID
GROUP BY E.FirstName, E.LastName
ORDER BY ROUND(SUM(O.Freight + OD.UnitPrice * OD.Quantity * (1-OD.Discount)), 2) DESC

**5.3 Ogranicz wynik z pkt 5.1 tylko do pracowników
a) którzy mają podwładnych
b) którzy nie mają podwładnych**

In [ ]:
SELECT E.FirstName, E.LastName,
    ROUND(SUM(O.Freight + OD.UnitPrice * OD.Quantity * (1-OD.Discount)), 2) AS TotalServed
FROM Employees AS E
LEFT OUTER JOIN Employees AS E2
ON E.EmployeeID = E2.ReportsTo
INNER JOIN Orders AS O
ON O.EmployeeID = E.EmployeeID
INNER JOIN [Order Details] AS OD
ON OD.OrderID = O.OrderID
WHERE E2.EmployeeID IS NOT NULL
GROUP BY E.FirstName, E.LastName

In [ ]:
SELECT E.FirstName, E.LastName,
    ROUND(SUM(O.Freight + OD.UnitPrice * OD.Quantity * (1-OD.Discount)), 2) AS TotalServed
FROM Employees AS E
LEFT OUTER JOIN Employees AS E2
ON E.EmployeeID = E2.ReportsTo
INNER JOIN Orders AS O
ON O.EmployeeID = E.EmployeeID
INNER JOIN [Order Details] AS OD
ON OD.OrderID = O.OrderID
WHERE E2.EmployeeID IS NULL
GROUP BY E.FirstName, E.LastName

**5.4 Zmodyfikuj rozwiązania z pkt 5.3 tak aby dla pracowników pokazać jeszcze datę
ostatnio obsłużonego zamówienia**

In [ ]:
SELECT E.FirstName, E.LastName,
    ROUND(SUM(O.Freight + OD.UnitPrice * OD.Quantity * (1-OD.Discount)), 2) AS TotalServed,
    MAX(O.OrderDate) AS LastOrderServed
FROM Employees AS E
LEFT OUTER JOIN Employees AS E2
ON E.EmployeeID = E2.ReportsTo
INNER JOIN Orders AS O
ON O.EmployeeID = E.EmployeeID
INNER JOIN [Order Details] AS OD
ON OD.OrderID = O.OrderID
WHERE E2.EmployeeID IS NOT NULL
GROUP BY E.FirstName, E.LastName